Retrieves meta data about all the samples from the NCBI data base using the entrezpy package

In [1]:
from pprint import pprint as pp
from tqdm.notebook import tqdm, trange
import time
import entrezpy.esummary.esummarizer
import re
import itertools
import covid_utilities as cu

In [2]:
with open(cu.DATA_PATH + r"\sampleNames.txt", "r") as f:
    sampleHeaders = f.readlines()
n_samples = len(sampleHeaders)

In [16]:
lookedup = []
lookedup_filepath = cu.COVID_PATH + r"\lookedup_samples.txt"
samples = {}
samples_filepath = cu.COVID_PATH + r"\samples.json"
grouping = 500

In [17]:
def get_data(headers, samples, lookedup):
    sampleNames = list(map(lambda h: re.findall(">(.*\.[0-9])", h)[0], headers))
    e = entrezpy.esummary.esummarizer.Esummarizer("L4project", "2408448g@student.gla.ac.uk")
    analyzer = e.inquire({'db' : 'nuccore', 'id' : sampleNames})
    summaries = analyzer.get_result().summaries
    for value in summaries.values():
        if "accessionversion" in value and value["accessionversion"] in sampleNames:
            types = value["subtype"].split("|")
            names = value["subname"].split("|")
            d = {}
            for i in range(len(types)):
                if types[i] == "collection_date":
                    d["collection_date"] = names[i]
                elif types[i] == "country":
                    d["country"] = names[i]
            samples[value["accessionversion"]] = d
    lookedup.extend(sampleNames)
    return samples, lookedup

In [ ]:
for headers in tqdm(list(zip(*[iter(sampleHeaders)]*grouping))):
    samples, lookedup = get_data(headers, samples, lookedup)
samples, lookedup = get_data(sampleHeaders[(n_samples - n_samples%grouping):], samples, lookedup)

In [25]:
with open(lookedup_filepath, "w") as f:
    f.write("\n".join(lookedup)+"\n")

In [26]:
cu.write_json(filename = samples_filepath, dictionary = samples)